In [67]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.data_preparation import *


In [68]:
bq_client = auth_gc_bigquery()
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.processed_train_data"
columns = '[spectrogram, labels]'

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [92]:
rows = get_output_query_bigquery(bq_client, table_id, limit=2000, columns="spectrogram")

Query returned 2000 results


In [93]:
def new_process_function(output):
    if output:
        spectrogram_bq, labels_bq, seconds_bq, duration_bq, podcast_name_bq = [], [], [], [], []
        for row in output:
            # Use row.field_name to access fields instead of indices
            spectrogram_bq.append(np.array(json.loads(row['spectrogram'])))
            labels_bq.append(row.get('label', None))  # Use get to handle missing fields
            seconds_bq.append(row.get('seconds', None)) if 'seconds' in row else seconds_bq.append(None)
            duration_bq.append(row.get('duration', None)) if 'duration' in row else duration_bq.append(None)
            podcast_name_bq.append(row.get('podcast_name', None)) if 'podcast_name' in row else podcast_name_bq.append(None)

        return spectrogram_bq, labels_bq, seconds_bq, duration_bq, podcast_name_bq

In [94]:
processed_columns = new_process_function(rows)

In [166]:
processed_columns

([array([[[0.48787862, 0.48787862, 0.48787862],
          [0.41997385, 0.41997385, 0.41997385],
          [0.10225592, 0.10225592, 0.10225592],
          ...,
          [0.06890602, 0.06890602, 0.06890602],
          [0.        , 0.        , 0.        ],
          [0.        , 0.        , 0.        ]],
  
         [[0.4725984 , 0.4725984 , 0.4725984 ],
          [0.39737853, 0.39737853, 0.39737853],
          [0.16807118, 0.16807118, 0.16807118],
          ...,
          [0.12842759, 0.12842759, 0.12842759],
          [0.07558747, 0.07558747, 0.07558747],
          [0.0420578 , 0.0420578 , 0.0420578 ]],
  
         [[0.4639298 , 0.4639298 , 0.4639298 ],
          [0.40034556, 0.40034556, 0.40034556],
          [0.15873365, 0.15873365, 0.15873365],
          ...,
          [0.23340735, 0.23340735, 0.23340735],
          [0.14881077, 0.14881077, 0.14881077],
          [0.16154107, 0.16154107, 0.16154107]],
  
         ...,
  
         [[0.10050831, 0.10050831, 0.10050831],
          [0.1

In [167]:
len(processed_columns[0])

2000

In [158]:
nans_count = 0
spec_column = processed_columns[0]

def check_for_nans(spec_column, nans_count):
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
                # print(f"NaNs found in array at index {idx}")
            nans_count += 1
    print(nans_count)

In [159]:
check_for_nans(spec_column, nans_count)

44


In [144]:
44/2000

0.022

In [168]:
spec_column = processed_columns[0]
label_column = processed_columns[1]

def check_for_and_delete_nans(spec_column, label_column, nans_count):
    cleaned_spec_column = []
    cleaned_label_column = []
    # New list to store non-NaN arrays
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
            nans_count += 1
        else:
            cleaned_spec_column.append(array)
            cleaned_label_column.append(label_column[idx])

    print(f"Total NaNs found: {nans_count}")
    return cleaned_spec_column, cleaned_label_column

In [174]:
cleaned_spec_column, cleaned_label_column = check_for_and_delete_nans(spec_column, label_column, nans_count)

Total NaNs found: 0


In [177]:
len(cleaned_spec_column)

1956

In [178]:
0.22*40000

8800.0

In [179]:
len(cleaned_label_column)

1956